In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from bayes_opt import BayesianOptimization

df_train = pd.read_csv("./regression.train", header=None, sep="\t")
y = df_train[0]
X = df_train.drop(0, axis=1)


df_test = pd.read_csv("./regression.test", header=None, sep="\t")
y_test = df_train[0]
X_test = df_train.drop(0, axis=1)

train_data = lgb.Dataset(data=X, label=y)
lgb_eval = lgb.Dataset(X_test, y_test, reference=train_data)

params = {'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': {'mae'},
         'num_leaves': 24,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.85,
        'max_depth': 5,
         'learning_rate': 0.02,
        'lambda_l1': 1,
        'lambda_l2': 1,
        'min_split_gain': 0.01,
        'min_child_weight': 10}

gbm = lgb.train(params, train_data, num_boost_round=300, valid_sets=lgb_eval, early_stopping_rounds=10)

y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print("the shape of y_pred", y_pred.shape)
print("the shape of y_truth", y_test.shape)
print(sum((y_pred > 0.5) == y_test))
                                    

[1]	valid_0's l1: 0.496232
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l1: 0.494701
[3]	valid_0's l1: 0.493282
[4]	valid_0's l1: 0.492212
[5]	valid_0's l1: 0.490486
[6]	valid_0's l1: 0.488977
[7]	valid_0's l1: 0.48754
[8]	valid_0's l1: 0.486135
[9]	valid_0's l1: 0.48488
[10]	valid_0's l1: 0.483417
[11]	valid_0's l1: 0.482388
[12]	valid_0's l1: 0.480743
[13]	valid_0's l1: 0.480059
[14]	valid_0's l1: 0.478711
[15]	valid_0's l1: 0.477369
[16]	valid_0's l1: 0.476113
[17]	valid_0's l1: 0.474724
[18]	valid_0's l1: 0.473703
[19]	valid_0's l1: 0.472454
[20]	valid_0's l1: 0.471162
[21]	valid_0's l1: 0.469788
[22]	valid_0's l1: 0.468557
[23]	valid_0's l1: 0.46733
[24]	valid_0's l1: 0.465964
[25]	valid_0's l1: 0.465133
[26]	valid_0's l1: 0.463945
[27]	valid_0's l1: 0.462769
[28]	valid_0's l1: 0.4614
[29]	valid_0's l1: 0.460348
[30]	valid_0's l1: 0.459473
[31]	valid_0's l1: 0.458381
[32]	valid_0's l1: 0.457143
[33]	valid_0's l1: 0.455867
[34]	valid_0's l1: 0.454575


[293]	valid_0's l1: 0.349278
[294]	valid_0's l1: 0.349078
[295]	valid_0's l1: 0.34893
[296]	valid_0's l1: 0.348836
[297]	valid_0's l1: 0.348658
[298]	valid_0's l1: 0.348423
[299]	valid_0's l1: 0.348304
[300]	valid_0's l1: 0.348182
Did not meet early stopping. Best iteration is:
[300]	valid_0's l1: 0.348182
the shape of y_pred (7000,)
the shape of y_truth (7000,)
5612


In [2]:
gbm = lgb.cv(params, train_data, nfold=3)

In [3]:
def bayes_parameter_opt_lgb(train_data, lgb_eval, init_round=15, opt_round=25,
                            random_seed=6, n_iterations=2000, learning_rate=0.05):

    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, 
                 max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'objective': 'regression','boosting_type': 'gbdt',
                  'num_iterations': n_iterations, 
                  'learning_rate':learning_rate, 'early_stopping_round':100, 
                  'metric':{'mae'}}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        gbm = lgb.cv(params, train_data, nfold=3, early_stopping_rounds=20)
        return -min(np.array(gbm['l1-mean'])) # -gbm.best_score['valid_0']['l1']
#         gbm = lgb.train(params, train_data, num_boost_round=3000,
#                         valid_sets=lgb_eval, early_stopping_rounds=10)
    

    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 10),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    

    return lgbBO

opt_params = bayes_parameter_opt_lgb(train_data, lgb_eval)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------


/home/wxk/anaconda3/envs/whale/lib/python3.6/site-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/wxk/anaconda3/envs/whale/lib/python3.6/site-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  1        | -0.3892   |  0.9098   |  0.6722   |  3.014    |  1.635    |  7.118    |  34.07    |  0.04432  |  42.73    |


/home/wxk/anaconda3/envs/whale/lib/python3.6/site-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/wxk/anaconda3/envs/whale/lib/python3.6/site-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  2        | -0.3904   |  0.9927   |  0.4068   |  3.959    |  1.587    |  7.84     |  46.65    |  0.008033 |  25.83    |


...

|  38       | -0.3851   |  0.993    |  0.8928   |  0.4976   |  2.691    |  9.654    |  41.41    |  0.0167   |  33.23    |


/home/wxk/anaconda3/envs/whale/lib/python3.6/site-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/wxk/anaconda3/envs/whale/lib/python3.6/site-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  39       | -0.386    |  0.8965   |  0.8891   |  0.01878  |  2.938    |  9.758    |  42.0     |  0.01407  |  41.4     |


/home/wxk/anaconda3/envs/whale/lib/python3.6/site-packages/lightgbm/engine.py:430: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/wxk/anaconda3/envs/whale/lib/python3.6/site-packages/lightgbm/engine.py:435: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


|  40       | -0.3868   |  0.8642   |  0.8834   |  0.2622   |  2.917    |  9.686    |  5.957    |  0.03216  |  33.21    |


In [5]:
sorted(opt_params.res, key = lambda x: -x['target'])

[{'target': -0.3850880748785738,
  'params': {'bagging_fraction': 0.9930206089804453,
   'feature_fraction': 0.8928020095326513,
   'lambda_l1': 0.4975860239948754,
   'lambda_l2': 2.691237823526113,
   'max_depth': 9.654239662932703,
   'min_child_weight': 41.41381242424631,
   'min_split_gain': 0.01670472500893779,
   'num_leaves': 33.22837367467527}},
 {'target': -0.38555201075603146,
  'params': {'bagging_fraction': 0.8054598352148743,
   'feature_fraction': 0.8975441212826849,
   'lambda_l1': 2.4953362486792763,
   'lambda_l2': 2.9942646682994223,
   'max_depth': 9.87136147644345,
   'min_child_weight': 48.88111169863027,
   'min_split_gain': 0.014240225148487579,
   'num_leaves': 27.397502172455894}},
 {'target': -0.3857388999653826,
  'params': {'bagging_fraction': 0.8584694037613704,
   'feature_fraction': 0.8740981518258352,
   'lambda_l1': 0.2674183897134913,
   'lambda_l2': 1.7850615152059315,
   'max_depth': 9.998325676492389,
   'min_child_weight': 40.01956056747095,
   'm